# Busiest Stations at Peak Time

In [50]:
-- temp tables for aggregates
create temporary table dates as

select *,
To_Char(start_time, 'YYYY') as year,
To_Char(start_time, 'MM') as month_no,
To_Char(start_time, 'ID') as day_no,
To_Char(start_time, 'HH24') as hour_no,
To_Char(start_time, 'Month') as month,
To_Char(start_time, 'Day') as day,
To_Char(start_time, 'HH12') || ':00 ' || To_Char(start_time, 'am') as hour
-- count(1) as riders

from bluebikes_2019
-- group by year, month_no, day_no, hour_no, month, day, hour
order by month_no, day_no, hour_no


Commands completed successfully

Total execution time: 00:00:08.336

In [1]:
select * from bluebikes_2019
order by random()
limit 10

(10 row(s) affected)

Total execution time: 00:00:00.983

bike_id,start_time,end_time,start_station_id,end_station_id,user_type,user_birth_year,user_gender
2428,2019-03-30T11:04:52.570000,2019-03-30T12:32:46.723000,6,97,Customer,1969,0
343,2019-06-14T16:06:20.745000,2019-06-14T16:34:16.695000,349,196,Customer,1969,0
901,2019-10-15T16:19:47.927000,2019-10-15T16:32:45.035000,176,100,Subscriber,1995,1
3194,2019-01-16T14:47:52.838000,2019-01-16T15:13:28.337000,145,68,Subscriber,1984,1
2607,2019-05-25T14:20:07.374000,2019-05-25T15:03:49.290000,142,107,Customer,1969,0
200,2019-08-05T19:33:01.671000,2019-08-05T19:39:59.681000,121,296,Subscriber,1982,1
5544,2019-12-10T07:36:53.863000,2019-12-10T08:01:07.759000,239,60,Subscriber,1988,1
5210,2019-10-05T15:01:17.021000,2019-10-05T15:25:12.489000,334,137,Customer,1996,1
2283,2019-12-10T19:19:58.310000,2019-12-10T19:31:36.063000,80,105,Subscriber,1980,1
2885,2019-07-09T06:49:01.939000,2019-07-09T06:54:35.065000,121,150,Subscriber,1995,2


In [51]:
select * from dates
order by random()
limit 10;

(10 row(s) affected)

Total execution time: 00:00:01.724

bike_id,start_time,end_time,start_station_id,end_station_id,user_type,user_birth_year,user_gender,year,month_no,day_no,hour_no,month,day,hour
4969,2019-10-01T17:14:11.550000,2019-10-01T17:19:48.732000,189,6,Customer,1969,0,2019,10,2,17,October,Tuesday,05:00 pm
2351,2019-02-11T08:25:33.832000,2019-02-11T08:54:57.301000,181,140,Subscriber,1977,1,2019,02,1,08,February,Monday,08:00 am
4020,2019-12-08T12:55:23.969000,2019-12-08T13:07:26.428000,46,12,Subscriber,1961,2,2019,12,7,12,December,Sunday,12:00 pm
5305,2019-09-22T14:52:37.610000,2019-09-22T15:12:47.534000,27,113,Customer,1969,0,2019,09,7,14,September,Sunday,02:00 pm
2977,2019-08-21T18:54:44.809000,2019-08-21T19:00:03.649000,12,46,Subscriber,1984,2,2019,08,3,18,August,Wednesday,06:00 pm
3566,2019-08-31T23:13:22.557000,2019-08-31T23:18:23.863000,85,98,Subscriber,1964,1,2019,08,6,23,August,Saturday,11:00 pm
4953,2019-08-24T15:21:13.882000,2019-08-24T15:48:07.182000,296,44,Subscriber,1992,1,2019,08,6,15,August,Saturday,03:00 pm
3673,2019-05-07T15:36:31.487000,2019-05-07T15:44:51.258000,84,107,Subscriber,1981,1,2019,05,2,15,May,Tuesday,03:00 pm
2698,2019-12-09T12:42:21.179000,2019-12-09T12:54:20.750000,318,107,Subscriber,1982,1,2019,12,1,12,December,Monday,12:00 pm
2464,2019-09-11T16:35:34.163000,2019-09-11T16:52:20.837000,272,411,Customer,1969,0,2019,09,3,16,September,Wednesday,04:00 pm


In [43]:
-- Top 5 busiest stations from any time.

select
a.month,
a.month_no,
a.day,
a.day_no,
a.hour,
a.hour_no,
sum(a.riders) as riders
from agg_2019 a
group by a.month, a.month_no, a.day, a.day_no, a.hour, a.hour_no
order by riders desc
limit 5

(5 row(s) affected)

Total execution time: 00:00:00.182

month,month_no,day,day_no,hour,hour_no,riders
October,10,Tuesday,2,05:00 pm,17,7827
July,07,Tuesday,2,05:00 pm,17,7752
August,08,Thursday,4,05:00 pm,17,7606
July,07,Monday,1,05:00 pm,17,7490
September,09,Monday,1,05:00 pm,17,7070


In [111]:
-- Find the stations that are going to have the highest influx at peak times


select 
end_station_id,
count(1) as bikes_in
-- sum(count(1)) over() as total_bikes,
-- avg(count(1)) over() as avg_bikes
from dates
WHERE 
(month_no, day_no, hour_no, start_station_id)
IN -- busiest stations from the busiest time
    (select
    month_no,
    day_no,
    hour_no,
    start_station_id
    from dates
    WHERE 
    (month_no, day_no, hour_no)
    IN -- Busiest time
        (select
        a.month_no,
        a.day_no,
        a.hour_no
        from agg_2019 a
        group by a.month, a.month_no, a.day, a.day_no, a.hour, a.hour_no
        order by sum(a.riders) desc
        limit 1
        -- offset 1
        )


    group by month_no, day_no, hour_no, start_station_id
    -- having count(1) > 150
    -- limit 3

    )


group by end_station_id
order by bikes_in desc
-- limit 3

(273 row(s) affected)

Total execution time: 00:00:01.585

end_station_id,bikes_in
67,80
68,61
74,59
189,59
107,54
80,47
52,43
76,41
73,39
61,38


In [101]:
    select
    month_no,
    day_no,
    hour_no,
    start_station_id,
    count(1) as riders,
    sum(count(1)) over() as total
    from dates
    WHERE 
    (month_no, day_no, hour_no)
    IN -- 5 busiest times
        (select
        a.month_no,
        a.day_no,
        a.hour_no
        from agg_2019 a
        group by a.month, a.month_no, a.day, a.day_no, a.hour, a.hour_no
        order by sum(a.riders) desc
        limit 5)


    group by month_no, day_no, hour_no, start_station_id
    having count(1) > 150

(25 row(s) affected)

Total execution time: 00:00:00.946

month_no,day_no,hour_no,start_station_id,riders,total
07,1,17,22,319,6151
07,1,17,24,204,6151
07,1,17,67,151,6151
07,1,17,107,377,6151
07,1,17,189,158,6151
07,2,17,22,333,6151
07,2,17,24,244,6151
07,2,17,67,169,6151
07,2,17,68,154,6151
07,2,17,107,419,6151
